# Google Cloud SQL for MySQL

> [Cloud SQL](https://cloud.google.com/sql) 是一个完全托管的关系数据库服务，提供高吞吐量、无缝集成和出色的可扩展性。它提供 PostgreSQL、MySQL 和 SQL Server 数据库引擎。利用 Cloud SQL 的 LangChain 集成，扩展您的数据库应用程序以构建支持 AI 的体验。

本笔记本将介绍如何使用 `Cloud SQL for MySQL` 通过 `MySQLVectorStore` 类来存储向量嵌入。

在 [GitHub](https://github.com/googleapis/langchain-google-cloud-sql-mysql-python/) 上了解有关该软件包的更多信息。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-mysql-python/blob/main/docs/vector_store.ipynb)

## 开始之前

要运行此笔记本，您需要执行以下操作：

* [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
* [启用 Cloud SQL Admin API](https://console.cloud.google.com/flows/enableapi?apiid=sqladmin.googleapis.com)
* [创建 Cloud SQL 实例](https://cloud.google.com/sql/docs/mysql/connect-instance-auth-proxy#create-instance)（版本必须 >= **8.0.36**，并且已将 **cloudsql_vector** 数据库标志配置为“On”）
* [创建 Cloud SQL 数据库](https://cloud.google.com/sql/docs/mysql/create-manage-databases)
* [向数据库添加用户](https://cloud.google.com/sql/docs/mysql/create-manage-users)

### 🦜🔗 库安装
安装集成库 `langchain-google-cloud-sql-mysql` 和嵌入服务库 `langchain-google-vertexai`。

In [ ]:
%pip install --upgrade --quiet langchain-google-cloud-sql-mysql langchain-google-vertexai

**仅限 Colab：**取消注释下面的单元格以重启内核或使用按钮重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 身份验证
您需要使用登录到此笔记本的 IAM 用户身份向 Google Cloud 进行身份验证，才能访问您的 Google Cloud 项目。

* 如果您是使用 Colab 运行此笔记本，请使用下面的单元格并继续。
* 如果您是使用 Vertex AI Workbench，请在此处查看设置说明 ([https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env))。

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便在此笔记本中使用 Google Cloud 资源。

如果您不知道项目 ID，请尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

## 基本用法

### 设置 Cloud SQL 数据库值
在 [Cloud SQL 实例页面](https://console.cloud.google.com/sql?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687) 中找到您的数据库值。

**注意：** MySQL 向量支持仅在版本 **>= 8.0.36** 的 MySQL 实例上可用。

对于现有实例，您可能需要执行一次 [自助维护更新](https://cloud.google.com/sql/docs/mysql/self-service-maintenance)，将维护版本更新到 **MYSQL_8_0_36.R20240401.03_00** 或更高版本。更新后，[配置您的数据库标志](https://cloud.google.com/sql/docs/mysql/flags)，将新的 **cloudsql_vector** 标志设置为“On”。

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
REGION = "us-central1"  # @param {type: "string"}
INSTANCE = "my-mysql-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
TABLE_NAME = "vector_store"  # @param {type: "string"}

### MySQLEngine 连接池

将 Cloud SQL 作为向量存储的一个要求和论点是 `MySQLEngine` 对象。`MySQLEngine` 会配置到您的 Cloud SQL 数据库的连接池，从而实现从您的应用程序成功连接，并遵循行业的最佳实践。

要使用 `MySQLEngine.from_instance()` 创建 `MySQLEngine`，您只需要提供 4 个要素：

1. `project_id`：Cloud SQL 实例所在 Google Cloud 项目的项目 ID。
1. `region`：Cloud SQL 实例所在的区域。
1. `instance`：Cloud SQL 实例的名称。
1. `database`：要连接到 Cloud SQL 实例上的数据库的名称。

默认情况下，[IAM 数据库身份验证](https://cloud.google.com/sql/docs/mysql/iam-authentication#iam-db-auth) 将被用作数据库身份验证的方法。此库使用属于[应用程序默认凭据 (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) 的 IAM 主体，该主体源自环境。

有关 IAM 数据库身份验证的更多信息，请参阅：

* [为 IAM 数据库身份验证配置实例](https://cloud.google.com/sql/docs/mysql/create-edit-iam-instances)
* [使用 IAM 数据库身份验证管理用户](https://cloud.google.com/sql/docs/mysql/add-manage-iam-users)

此外，还可以使用[内置数据库身份验证](https://cloud.google.com/sql/docs/mysql/built-in-authentication)，即使用用户名和密码访问 Cloud SQL 数据库。只需为 `MySQLEngine.from_instance()` 提供可选的 `user` 和 `password` 参数即可：

* `user`：用于内置数据库身份验证和登录的数据库用户。
* `password`：用于内置数据库身份验证和登录的数据库密码。

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLEngine

engine = MySQLEngine.from_instance(
    project_id=PROJECT_ID, region=REGION, instance=INSTANCE, database=DATABASE
)

### 初始化表
`MySQLVectorStore` 类需要一个数据库表。`MySQLEngine` 类有一个名为 `init_vectorstore_table()` 的辅助方法，可以用来为您创建具有适当架构的表。

In [ ]:
engine.init_vectorstore_table(
    table_name=TABLE_NAME,
    vector_size=768,  # Vector size for VertexAI model(textembedding-gecko@latest)
)

### 创建嵌入类实例

您可以使用任何 [LangChain embeddings 模型](/docs/integrations/text_embedding/)。
您可能需要启用 Vertex AI API 来使用 `VertexAIEmbeddings`。

我们建议在生产环境中固定嵌入模型的版本，了解更多有关 [文本嵌入模型](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text-embeddings) 的信息。

In [ ]:
# enable Vertex AI API
!gcloud services enable aiplatform.googleapis.com

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

### 初始化默认的 MySQLVectorStore

要初始化 `MySQLVectorStore` 类，您只需要提供 3 个要素：

1.  `engine` - `MySQLEngine` 引擎的实例。
1.  `embedding_service` - LangChain 嵌入模型的实例。
1.  `table_name`：Cloud SQL 数据库中用作向量存储的表名。

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLVectorStore

store = MySQLVectorStore(
    engine=engine,
    embedding_service=embedding,
    table_name=TABLE_NAME,
)

### 添加文本

In [ ]:
import uuid

all_texts = ["Apples and oranges", "Cars and airplanes", "Pineapple", "Train", "Banana"]
metadatas = [{"len": len(t)} for t in all_texts]
ids = [str(uuid.uuid4()) for _ in all_texts]

store.add_texts(all_texts, metadatas=metadatas, ids=ids)

### 删除文本

通过 ID 从向量存储中删除向量。

In [ ]:
store.delete([ids[1]])

### 搜索文档

In [17]:
query = "I'd like a fruit."
docs = store.similarity_search(query)
print(docs[0].page_content)

Pineapple


### 按向量搜索文档

也可以使用 `similarity_search_by_vector` 来搜索与给定嵌入向量相似的文档，该函数接受一个嵌入向量作为参数而不是字符串。

In [ ]:
query_vector = embedding.embed_query(query)
docs = store.similarity_search_by_vector(query_vector, k=2)
print(docs)

[Document(page_content='Pineapple', metadata={'len': 9}), Document(page_content='Banana', metadata={'len': 6})]


### 添加索引
通过应用向量索引来加速向量搜索查询。了解更多关于 [MySQL 向量索引](https://github.com/googleapis/langchain-google-cloud-sql-mysql-python/blob/main/src/langchain_google_cloud_sql_mysql/indexes.py) 的信息。

**注意：** 对于 IAM 数据库身份验证（默认用法），IAM 数据库用户需要被授权以下权限，才能对向量索引拥有完全控制权。这些权限需要由具有特权的数据库用户授予。

```
GRANT EXECUTE ON PROCEDURE mysql.create_vector_index TO 'your_iam_db_user'@'%';
GRANT EXECUTE ON PROCEDURE mysql.alter_vector_index TO 'your_iam_db_user'@'%';
GRANT EXECUTE ON PROCEDURE mysql.drop_vector_index TO 'your_iam_db_user'@'%';
GRANT SELECT ON mysql.vector_indexes TO 'your_iam_db_user'@'%';
```

In [ ]:
from langchain_google_cloud_sql_mysql import VectorIndex

store.apply_vector_index(VectorIndex())

### 移除索引

In [ ]:
store.drop_vector_index()

## 高级用法

### 使用自定义元数据创建 MySQLVectorStore

向量存储可以利用关系数据来过滤相似性搜索。

创建一个表和带有自定义元数据列的 `MySQLVectorStore` 实例。

In [ ]:
from langchain_google_cloud_sql_mysql import Column

# set table name
CUSTOM_TABLE_NAME = "vector_store_custom"

engine.init_vectorstore_table(
    table_name=CUSTOM_TABLE_NAME,
    vector_size=768,  # VertexAI model: textembedding-gecko@latest
    metadata_columns=[Column("len", "INTEGER")],
)


# initialize MySQLVectorStore with custom metadata columns
custom_store = MySQLVectorStore(
    engine=engine,
    embedding_service=embedding,
    table_name=CUSTOM_TABLE_NAME,
    metadata_columns=["len"],
    # connect to an existing VectorStore by customizing the table schema:
    # id_column="uuid",
    # content_column="documents",
    # embedding_column="vectors",
)

### 使用元数据过滤器搜索文档

在处理文档之前，对文档进行筛选会很有帮助。

例如，可以使用 `filter` 参数根据元数据筛选文档。

In [ ]:
import uuid

# add texts to the vector store
all_texts = ["Apples and oranges", "Cars and airplanes", "Pineapple", "Train", "Banana"]
metadatas = [{"len": len(t)} for t in all_texts]
ids = [str(uuid.uuid4()) for _ in all_texts]
custom_store.add_texts(all_texts, metadatas=metadatas, ids=ids)

# use filter on search
query_vector = embedding.embed_query("I'd like a fruit.")
docs = custom_store.similarity_search_by_vector(query_vector, filter="len >= 6")

print(docs)

[Document(page_content='Pineapple', metadata={'len': 9}), Document(page_content='Banana', metadata={'len': 6}), Document(page_content='Apples and oranges', metadata={'len': 18}), Document(page_content='Cars and airplanes', metadata={'len': 18})]
